Перед отправкой почистить. Здесь куча хлама. =D

#### Оглавление:
1. [Объявление основных функций](#StimViz)
2. [Выбор файлов и выполнение преобразования](#Open_files)
3. [Объявление класса Point, методов ГА и функции получения проекции координат](#GA_preparation)
4. [Выполнение ГА](#GA)
5. Визуализация результата через StimViz


 ## <a name="StimViz"></a> **I.** Объявление основных функций

In [2]:
from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti
import numpy as np
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.tracking import streamline
import copy
import pickle
import math
import simnibs
from simnibs import sim_struct, run_simnibs
from simnibs.msh import (surface,
                        mesh_io,
                        read_msh)
import random

In [3]:
def my_deriv(t,line,h=1):
    if t==0:
        return (-3*line[t]+4*line[t+1]-line[t+2])/(2*h)
    elif t==len(line)-1:
        return (line[t-2]-4*line[t-1]+3*line[t])/(2*h)
    else:
        return (line[t+1]-line[t-1])/(2*h)

def load_elems(nodes,elems):

    import numpy as np

    elems = elems[elems[:,3]!= -1,:]
    # Computing rectangles
    tmp = nodes[elems-1,:]
    elems_min = tmp.min(axis=1)
    elems_max = tmp.max(axis=1)
    tmp = 0
    sizes = (elems_max-elems_min).max(axis=0)
    # It is the index to reduce the elements to check
    order_min = np.argsort(elems_min[:,0])
    return {"Nodes":nodes, "Elems":elems, "El_min":elems_min, "El_max":elems_max, "Sizes":sizes,"Order_min":order_min}

# a function to get e-field vector at a given position [x,y,z]
def get_field(ttt, point, my_field):
    best_ttt=get_ttrd(ttt,point)
    return np.dot(my_field[best_ttt['Nodes']-1].T, best_ttt['Weights'])

def get_ttrd(loaded_elems,point):
    import numpy as np
    # Just to use names I have used before
    nodes = loaded_elems["Nodes"]
    elems = loaded_elems["Elems"]
    elems_min = loaded_elems["El_min"]
    elems_max = loaded_elems["El_max"]
    sizes = loaded_elems["Sizes"]
    order_min = loaded_elems["Order_min"]
    
    # Binary search to reduce the iterating points from 4mln to around 200k.
    r = np.searchsorted(elems_min[:,0],point[0],side='left',sorter=order_min)
    l = np.searchsorted(elems_min[:,0],point[0] - sizes[0],side='right',sorter=order_min)
    # Projection the data to only these points
    e_max = elems_max[order_min[l:r]]
    e_min = elems_min[order_min[l:r]]
    
    # Checks which ttrds are possible to contain the point
    potential_ttrds = order_min[l:r][(point[0] <= e_max[:,0]) & (e_min[:,1]<= point[1]) & (point[1] <= e_max[:,1]) & (e_min[:,2]<= point[2]) & (point[2] <= e_max[:,2])]
    
    # It checks if the ttrd contains the point
    def check_ttrd(ttrd, point):
        coord = np.column_stack((ttrd[1,:]-ttrd[0,:],ttrd[2,:]-ttrd[0,:],ttrd[3,:]-ttrd[0,:]))
        coord = np.linalg.inv(coord).dot(point-ttrd[0,:])
        return coord.min() >= 0 and coord.sum() <= 1
    # It checks if the ttrd with num ttrdNum contains the point
    def check_ttrd_byNum(ttrdNum, point):
        ttrd = nodes[elems[ttrdNum]-1]
        return check_ttrd(ttrd,point)
    
    # Just takes all ttrds that contain points
    nodeIndices = elems[[x for x in potential_ttrds if check_ttrd_byNum(x,point)]][0]; 
    ns = nodes[nodeIndices-1]

    norms = np.sum((ns-point)**2,axis=-1)**0.5
    weights = 1/(norms+1e-10)
    weights = weights / weights.sum()
    
    return {"Nodes":nodeIndices,"Weights":weights}


# a function to calculate directional derivatives of the effective field at a given point [x,y,z]
def deriv_e_field(coordinates, e_field_nodes, LSD, ttt):
   
    step=0.05

    x1=coordinates[0]
    y1=coordinates[1]
    z1=coordinates[2]
    x0=coordinates[0]-step
    x2=coordinates[0]+step
    y0=coordinates[1]-step
    y2=coordinates[1]+step
    z0=coordinates[2]-step
    z2=coordinates[2]+step

    f_x0_y1_z1=np.dot(get_field(ttt,np.asarray([x0,y1,z1]), e_field_nodes), LSD)
    f_x2_y1_z1=np.dot(get_field(ttt,np.asarray([x2,y1,z1]), e_field_nodes), LSD)
    f_x1_y1_z1=np.dot(get_field(ttt,np.asarray([x1,y1,z1]), e_field_nodes), LSD)
    f_x1_y0_z1=np.dot(get_field(ttt,np.asarray([x1,y0,z1]), e_field_nodes), LSD)
    f_x1_y2_z1=np.dot(get_field(ttt,np.asarray([x1,y2,z1]), e_field_nodes), LSD)
    f_x1_y1_z0=np.dot(get_field(ttt,np.asarray([x1,y1,z0]), e_field_nodes), LSD)
    f_x1_y1_z2=np.dot(get_field(ttt,np.asarray([x1,y1,z2]), e_field_nodes), LSD)
    
    gradx=my_deriv(1,[f_x0_y1_z1,f_x1_y1_z1,f_x2_y1_z1], step)
    grady=my_deriv(1,[f_x1_y0_z1,f_x1_y1_z1,f_x1_y2_z1], step)
    gradz=my_deriv(1,[f_x1_y1_z0,f_x1_y1_z1,f_x1_y1_z2], step)
    
    return np.dot([gradx, grady, gradz], LSD)

def simulation(fnamehead, pathfem, pos_centre=[-74.296158, -10.213354, 28.307243], pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'
    
    # Initialize a coil position
    pos = tmslist.add_position()
    pos.centre = pos_centre # Place the coil over
    pos.pos_ydir = pos_ydir # Point the coil towards
    pos.distance = distance # Distance between coil and head
    pos.didt = current_change # Rate of change of current in the coil, in A/s.
    run_simnibs(s)

def calc_TMS_effects(x, y, z): # (change_TMS_effects)
    
    l1 = 2  # membrane space constant 2mm
    l2 = l1**2
    
    #position = [x-256/2, y-256/2, z-256/2]  # -256/2 because of a freesurfer RAS coordinate system
    current_out_dir = out_dir+str(x)+'_'+str(y)+'_'+str(z)

    simulation(mesh_path, current_out_dir, pos_centre=[x,y,z]) #pos_centre=position

    mesh_file = current_out_dir+'/'+subject_name+'_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
    field_mesh = read_msh(mesh_file)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)

    effective_field = copy.deepcopy(new_streams_T1_array)

    for stream in range(len(new_streams_T1_array)):
        my_steam = copy.deepcopy(new_streams_T1_array[stream])
        print('starting _'+str(stream)+' out of '+str(len(new_streams_T1_array)))
        for t in range(len(my_steam[:, 0])):
            # -256/2 because of a freesurfer RAS coordinate system
            x = my_steam[t, 0]-256/2
            y = my_steam[t, 1]-256/2
            z = my_steam[t, 2]-256/2
            xyz = np.asarray([x, y, z])

            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            effective_field[stream][t, 0] = l1*np.dot(field_vector_xyz, streams_array_derivative[stream][t, :])
            effective_field[stream][t, 1] = l2*deriv_e_field(xyz, field_at_nodes, streams_array_derivative[stream][t, :], ttt)
            effective_field[stream][t, 2] = effective_field[stream][t, 0] + effective_field[stream][t, 1]
            
    with open(current_out_dir+'/'+subject_name+'_effective_field.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()
    # Поиск максимума среди всех точек:
    return max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])



## <a name="Open_files"></a>**II.** Выбор файлов и выполнение преобразования


In [5]:
# Ввод данных
tractography_file='Data/tracts.trk'

sft = load_trk(tractography_file, tractography_file)
streams = sft.streamlines
streams_array = np.asarray(streams)
print('imported tractography data:'+tractography_file)

fname_T1 = 'Data/T1fs_conform.nii.gz'#input("Please, specify the T1fs_conform image that has been generated during head meshing procedure. ")
data_T1, affine_T1 = load_nifti(fname_T1)

fname_FA = 'Data/dti_fa.nii' #input("Please, specify the FA image. ")
data_FA, affine_FA = load_nifti(fname_FA)
print('loaded T1fs_conform.nii and FA images')

mesh_path = 'Data/subject.msh' #input("Please, specify the head mesh file. ")

last_slach = max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
subject_name = mesh_path[last_slach:-4]


# specify the directory where you would like to save your simulation results
# input example:'/home/Example_data/Output'
out_dir = 'Output' #input("Please, specify the directory where you would like to save your simulation results. ")
out_dir = out_dir+'/simulation_at_pos_'

imported tractography data:Data/tracts.trk
loaded T1fs_conform.nii and FA images


In [6]:
# To avoid recalculating the code cell below each time.

with open('../new_streams_T1_array.txt', 'rb') as f:
    new_streams_T1_array = pickle.load(f)
f.close()

with open('../streams_array_derivative.txt', 'rb') as f:
    streams_array_derivative = pickle.load(f)
f.close()

In [ ]:
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1, data_FA, affine_FA)
print('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                    affine_T1, affine_FA,
                    starting_affine=starting_affine)
print('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0, 
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images.
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                         affine_T1, affine_FA,
                         starting_affine=starting_affine)

print('calculated Affine 3D transform')

identity = np.eye(4)

inv_affine_FA = np.linalg.inv(affine_FA)
inv_affine_T1 = np.linalg.inv(affine_T1)
inv_affine = np.linalg.inv(affine.affine)

# transforming streamlines to FA space
new_streams_FA = streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array = np.asarray(new_streams_FA)

T1_to_FA = np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1))
FA_to_T1 = np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
global new_streams_T1_array
new_streams_T1_array = np.asarray(new_streams_T1)

# calculating amline derivatives along the streamlines to get the local orientation of the streamlines
global streams_array_derivative
streams_array_derivative = copy.deepcopy(new_streams_T1_array)

print('calculating streamline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam = new_streams_T1_array[stream]
    for t in range(len(my_steam[:, 0])):
        streams_array_derivative[stream][t, 0] = my_deriv(t, my_steam[:, 0])
        streams_array_derivative[stream][t, 1] = my_deriv(t, my_steam[:, 1])
        streams_array_derivative[stream][t, 2] = my_deriv(t, my_steam[:, 2])
        deriv_norm = np.linalg.norm(streams_array_derivative[stream][t, :])
        streams_array_derivative[stream][t, :] = streams_array_derivative[stream][t, :]/deriv_norm

-----------
## <a name="GA_preparation"></a> **III.** Объявление класса Point, методов ГА и функции получения проекции координат
class Point, get_parents_id(), crossover(parents_id), get_coords_from_angles(alfa, beta)

In [42]:
class Point:
    def __init__(self, angles, proj_coords):
        self.angles=angles
        self.coords=proj_coords
        
        
    def __lt__(self, other):
        return self.fitness > other.fitness
    def mutation(self, recalc_fitness=False):
        self.angles[random.getrandbits(1)]=random.uniform(0, 180)
        self.coords= get_coords_from_angles(angles).tolist()
        if recalc_fitness:
            self.fitness=change_TMS_effects(*self.coords)
    def calc_fitness(self):
        try:
            x,y,z=self.coords
            path_to_possible_effective_field=out_dir+str(x)+'_'+str(y)+'_'+str(z)+'/'+subject_name+'_effective_field.txt'
            
            # if simulation on this coordinates somehow was done earlier
            if exists(path_to_possible_effective_field):
                with open(path_to_possible_effective_field, 'rb') as f:
                    effective_field = pickle.load(f)
                f.close()
                self.fitness=max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])
            
            # normal way
            else:
                self.fitness=calc_TMS_effects(*self.coords)
        except Exception as ex:
            print('EROR_EROR_SO_SAD_EROR', self.coords)
            print(ex)

In [8]:
# Определение родительской пары
def get_parents_id():   
    
    def get_parent(fit_values):
        randnum=random.random()
        fitness_sum = fit_values.sum() #sum(fit_values)
        fit_values= fit_values/fitness_sum
        cdf=np.concatenate([np.array([0]), fit_values.cumsum()])
        for i in range(len(cdf)-1):
            if((randnum>cdf[i])&(randnum<=cdf[i+1])):
                return i
            
    #generation.sort()
    fit_values = np.array([x.fitness for x in  generation])
    parent_id_a= get_parent(fit_values)
    parent_id_b= get_parent(np.delete(fit_values, parent_id_a))
    if parent_id_b>=parent_id_a:
        parent_id_b+=1;
    return(parent_id_a, parent_id_b)

In [9]:
# Скрещивание
# Через арифметический кроссовер (по сути морфинг)
def crossover(parents_id): 
    angles_a = generation[parents_id[0]].angles
    angles_b = generation[parents_id[1]].angles
    w = random.random()
    print(w)
    angles_c1 = [w * angles_a[0] + (1-w) * angles_b[0],
                 w * angles_a[1] + (1-w) * angles_b[1]]
    angles_c2 = [w * angles_b[0] + (1-w) * angles_a[0],
                 w * angles_b[1] + (1-w) * angles_a[1]]
    return angles_c1, angles_c2

In [10]:
mesh_struct = mesh_io.read_msh(mesh_path)
scalp=surface.Surface(mesh_struct, [5,1005])

def get_coords_from_angles(alfa, beta):
    # (1) Преобразование в трехмерную систему координат. x,y,z ∈ [0,255]
    r=256/2
    a= math.radians(alfa)
    b= math.radians(beta)
    
    x= r * math.sin(a) * math.cos(b) +r
    y= r * math.cos(a) +r
    z= r * math.sin(a) * math.sin(b) +r
    
    coords=np.array((x,y,z))
    coords-=256/2
    
    # (2) Получение проекции координаты на голову
    proj_coords,_=scalp.projectPoint(coords)
    return proj_coords

## <a name="GA"></a>**IV.** Выполнение ГА:

----
## IV_1. Вычисления с использованием multiprocessing
----

### Многопроцессорность

In [11]:
import multiprocessing
from os.path import exists
import time

In [12]:
class Consumer(multiprocessing.Process):

    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        pname = self.name
            
        while True:
            point = self.task_queue.get()
            
            if point is None: # poison pill
                print('Exiting %s...' % pname)
                self.task_queue.task_done()
                break
                
            print('{0} processing task: {1}'.format(pname, point.coords))
            time.sleep(1)
            point.calc_fitness()
            
            self.task_queue.task_done()
            self.result_queue.put(point)

----
### Выполнение ГА
(I)
    1. Generate random list with angles (angles_list)
    2. Calculate projected coordinates and effect.
(II)
    3. Create new generation and calculate effect
    4. Repeat from II

In [13]:
# Settings
number_of_generations=10
number_of_children=6
individs_in_generation=15
mutation_start=2

In [14]:
# Only to visualize progress
# Also need to comment one line ("prg_bar.value+=1") code cell below
# if there is no need to visualize progress
from ipywidgets import IntProgress
from IPython.display import display
prg_bar = IntProgress(min = 0, max = number_of_generations) # Создаем прогрессбар
display(prg_bar)

IntProgress(value=0, max=10)

In [44]:
# GA with multiprocessing
generation=[]
points=[]
steps_with_unchanged_leader=0
leader=0

for step in range(0, number_of_generations):
    if step==0:
        # Create angles for first generation
        angles_list=[[random.uniform(0,180) for i in range(2)]
                     for x in range(10)]
        points=[Point(angles, get_coords_from_angles(*angles).tolist())
                for angles in angles_list]
    else:
        points.clear()
        # Generate angles for children by crossover
        while (len(points)<number_of_children):
            children_angles=crossover(get_parents_id())
            
            for child_angles in children_angles:
                child_coords=get_coords_from_angles(*child_angles).tolist()
                # uniqueness test
                while child_coords in [point.coords for point in points]:
                    child_angles[random.getrandbits(1)]=random.uniform(0, 180)
                    child_coords=get_coords_from_angles(*child_angles).tolist()
                
                points.append(Point(child_angles, child_coords))
    # muation
    if steps_with_unchanged_leader>=mutation_start:
        del(generation[randint(3,individs_in_generation)])
        angles=[random.uniform(0,180) for i in range(2)]
        coords=get_coords_from_angles(angles())
        points.append(Point())
    
    
    # multiprocessing calculation
    if __name__ == '__main__':
        tasks = multiprocessing.JoinableQueue()
        results = multiprocessing.Queue()
        # spawning consumers
        n_consumers = 2 #multiprocessing.cpu_count()
        print('Spawning %i consumers...' % n_consumers)
        consumers = [Consumer(tasks, results) for i in range(n_consumers)]

        for consumer in consumers:
            consumer.start()

        # enqueueing jobs
        for point in points:
            tasks.put(point)
        # poison pill
        for i in range(n_consumers):
            tasks.put(None)
        
        tasks.join()
        
        for i in range(len(points)):
            generation.append(results.get())
    generation.sort()
    generation= generation[:individs_in_generation]
    
    if leader==generation[0].fitness:
        steps_with_unchanged_leader+=1
    else:
        steps_with_unchanged_leader=0
        leader=generation[0].fitness
    
    print('===== Generation {0} ====='.format(step))
    prg_bar.value+=1 # <--comment this line if there is no need to visualize progress
    for point in generation:
        print(point.coords, point.fitness)
    print('==========================')

0.05397578840113082
0.40846786967379356
0.7890914349319982
Spawning 2 consumers...
Consumer-17 processing task: [42.292686, -56.308277, 64.647308]
Consumer-18 processing task: [-68.691261, -36.813747, 46.078426]
-----


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.292686_-56.308277_64.647308
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.292686_-56.308277_64.647308
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-

[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_42.292686_-56.308277_64.647308/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_42.292686_-56.308277_64.647308/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.292686_-56.308277_64.647308/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_42.292686_-56.308277_64.647308/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-17 processing task: [-27.737192, -38.433205, 77.129356]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-27.737192_-38.433205_77.129356
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-27.737192_-38.433205_77.129356


...

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-67.647385_-45.913666_44.162548
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-67.647385_-45.913666_44.162548
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_-70.249039_-35.677868_41.018021/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |1.47e+00 V/m |1.03e+00 V/m |5.86e-01 V/m |
|normE |1.47e+00 V/m |1.03e+00 V/m |5.86e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.12e+03 mm³ |1.59e+04 mm³ |
|normE |4.12e+03 mm³ |1.59e+04 mm³ |

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_-70.249039_-35.677868_41.

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-32 processing task: [-59.040722, 53.439301, 31.245174]
Exiting Consumer-31...


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-59.040722_53.439301_31.245174
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-59.040722_53.439301_31.245174
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-59.040722_53.439301_31.245174
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-59.040722_53.439301_31.245174
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Be

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-32...
--2--
--3--
===== Generation 9 =====
[-53.106174, -35.167816, 62.262871] 3.288109050371429
[-56.042183, -33.389053, 60.28146] 3.236452937706372
[-58.227116, -35.689987, 58.149925] 3.028075014245654
[-45.451393, -36.077049, 67.414757] 2.80218333449395
[-61.187836, -33.306942, 56.082043] 2.625348932483163
[-62.366817, -32.924835, 55.012039] 2.5838459731094026
[-67.52607, -33.200184, 48.273495] 2.552008056597893
[-67.52607, -33.200184, 48.273495] 2.552008056597893
[-69.147957, -35.355515, 44.82407] 2.5184051261825426
[-69.147957, -35.355515, 44.82407] 2.5184051261825426
[-70.249039, -35.677868, 41.018021] 2.5049962699608983
[-70.249039, -35.677868, 41.018021] 2.5049962699608983
[-69.975845, -36.508789, 42.539047] 2.504842338110123
[-69.975845, -36.508789, 42.539047] 2.504842338110123
[-41.133171, -36.141922, 70.394997] 2.4969298912153484


In [47]:
# 10 steps result
print(generation[0].fitness, generation[0].coords)

3.288109050371429 [-53.106174, -35.167816, 62.262871]


----

In [46]:
for x in generation:
    print(x.fitness, x.angles, x.coords)

3.288109050371429 [108.05353447846804, 129.60713581564096] [-53.106174, -35.167816, 62.262871]
3.236452937706372 [107.72665215678462, 131.06128092556065] [-56.042183, -33.389053, 60.28146]
3.028075014245654 [107.91244203298189, 133.99441970436095] [-58.227116, -35.689987, 58.149925]
2.80218333449395 [109.12045990907012, 123.97948505478328] [-45.451393, -36.077049, 67.414757]
2.625348932483163 [106.91482910827858, 135.61340013284178] [-61.187836, -33.306942, 56.082043]
2.5838459731094026 [106.76898574918779, 136.66840412214566] [-62.366817, -32.924835, 55.012039]
2.552008056597893 [104.47572197004494, 147.2242072442093] [-67.52607, -33.200184, 48.273495]
2.552008056597893 [103.64108502403067, 147.74495362992434] [-67.52607, -33.200184, 48.273495]
2.5184051261825426 [103.46016762533586, 151.25561661335706] [-69.147957, -35.355515, 44.82407]
2.5184051261825426 [103.62033240695118, 152.66231033036232] [-69.147957, -35.355515, 44.82407]
2.5049962699608983 [102.92102802740325, 155.4495425168

In [45]:
with open('../generation_10_steps.txt', 'wb') as f:
    pickle.dump(generation, f)
f.close()

In [43]:
with open('../generation.txt', 'rb') as f:
    generation = pickle.load(f)
f.close()

-------------------